In [1]:
import random
from enum import auto, Enum
from subprocess import run
from textwrap import dedent
from typing import Iterable, Optional


class GameStatus(Enum):
    PLAYING = auto()
    LOSE = auto()
    WIN = auto()


class MineBoard:
    status: GameStatus = GameStatus.PLAYING
    w: int
    h: int
    board: list[list[int]]
    cells_to_open: int

    def __init__(self, w: int, h: int, k: int) -> None:
        # Create a new board with size w x h
        self.w = w
        self.h = h
        self.board = [[0 for _ in range(w)] for _ in range(h)]
        self.allocate_mines(w, h, k)
        self.cells_to_open = w * h - k

    def allocate_mines(self, w: int, h: int, num_of_mines: int) -> None:
        alloc_indexes = self.get_random_pos(w * h, num_of_mines)
        for i in alloc_indexes:
            self.set_mine(int(i / w), i % h)
            self.set_adjacent_mines(int(i / w), i % h)

    def __str__(self) -> str:
        return "\n".join(
            [
                " " * 7 + "".join(map(lambda x: "{:^7d}".format(x + 1), range(self.w))),
                " " * 7 + "-" * (self.w * 7),
                "\n".join(
                    (
                        "{:^7d}".format(i + 1)
                        + "|"
                        + " |".join(
                            list(map(lambda x: "{:^5s}".format(self.display(x)), row))
                        )
                        + " | "
                    )
                    + "\n"
                    + (" " * 7 + "-" * (self.w * 7))
                    for (i, row) in enumerate(self.board)
                ),
            ]
        )

    def click(self, row: int, col: int) -> None:
        value = self.reveal(row, col)
        if value:
            self.cells_to_open -= 1
            if self.cells_to_open == 0:
                self.status = GameStatus.WIN
            if self.has_mine(row, col):
                self.status = GameStatus.LOSE
            elif self.is_blank(row, col):
                for dr in range(row - 1, row + 2):
                    for dc in range(col - 1, col + 2):
                        self.click(dr, dc)

    def flag(self, row: int, col: int) -> None:
        if self.is_valid_cell(row, col) and self.is_hidden(row, col):
            self.toggle_flag(row, col)

    def is_valid_cell(self, row: int, col: int) -> bool:
        return 0 <= row < self.h and 0 <= col < self.w

    def get_random_pos(self, n: int, k: int) -> Iterable[int]:
        res: list[int] = []
        remains = list(range(n))
        while k > 0:
            r = random.randint(0, len(remains) - 1)
            res.append(r)
            del remains[r]
            k -= 1
        return res

    # Convention for cell values:
    #    - 0 : Hidden Blank
    #    - 10 : Revealed Blank
    #    - -1 : Hidden Bomb
    #    - 9 : Revealed Bomb
    #    - 1 ~ 8 : number of adjacent bomb (hidden)
    #    - 11 ~ 18 : adjacent bomb (revealed)
    #    - x + 100 : Flagged

    def set_mine(self, row: int, col: int) -> None:
        self.board[row][col] = -1

    def set_adjacent_mines(self, row: int, col: int) -> None:
        for dr in range(row - 1, row + 2):
            for dc in range(col - 1, col + 2):
                if self.is_valid_cell(dr, dc) and not self.has_mine(dr, dc):
                    self.board[dr][dc] += 1

    def toggle_flag(self, row: int, col: int) -> None:
        if self.is_flagged(row, col):
            self.board[row][col] -= 100
        else:
            self.board[row][col] += 100

    # Open a cell and return its value
    def reveal(self, row: int, col: int) -> Optional[int]:
        if not self.is_valid_cell(row, col) or not self.is_hidden(row, col):
            return None
        if self.is_flagged(row, col):
            self.toggle_flag(row, col)
        self.board[row][col] += 10
        return self.board[row][col]

    def is_hidden(self, row: int, col: int) -> bool:
        return self.board[row][col] < 9

    def has_mine(self, row: int, col: int) -> bool:
        return self.board[row][col] % 10 == 9

    def is_blank(self, row: int, col: int) -> bool:
        return self.board[row][col] % 10 == 0

    def is_over(self) -> bool:
        return self.win_game() or self.lose_game()

    def lose_game(self) -> bool:
        return self.status == GameStatus.LOSE

    def win_game(self) -> bool:
        return self.status == GameStatus.WIN

    def is_flagged(self, row: int, col: int) -> bool:
        return self.board[row][col] > 90

    def reveal_all(self) -> None:
        for i in range(len(self.board)):
            for j in range(len(self.board[0])):
                self.reveal(i, j)

    def display(self, ip: int) -> str:
        if ip > 90:
            return "F"
        if ip == 9:
            return "*"
        if ip == 10:
            return "."
        if ip > 10:
            return str(ip - 10)
        return ""


def cls() -> None:
    run(["/usr/bin/env", "clear"], check=True)


def play() -> None:
    w = int(input("Enter width of board: "))
    h = int(input("Enter height of board: "))
    m = int(input("Enter number of mines : "))
    while m >= w * h - 1:
        m = int(input("Too many mines. Enter again : "))

    game = MineBoard(w, h, m)

    while not game.is_over():
        cls()
        print(game)

        command = next_command()

        splits = command.split(" ")
        row = int(splits[0]) - 1
        col = int(splits[1]) - 1

        if command[-1] == "F":
            game.flag(row, col)
        else:
            game.click(row, col)

    game.reveal_all()
    cls()
    print(game)

    if game.lose_game():
        print("You lose !!")
    elif game.win_game():
        print("You win !!. Congradulations !!")


def next_command() -> str:
    instruction = dedent(
        """\
            Commands :
            <row> <col> : open cell
            <row> <col> F : flag cell
            q : quit
        """
    )
    return input(instruction).strip()


if __name__ == "__main__":
    play()

ValueError: invalid literal for int() with base 10: ''